In [6]:
import tensorflow as tf
import numpy as np 
import pandas as pd
tf.compat.v1.executing_eagerly()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
x_data = Candledf.index
y_data = Candledf.close
x_data,y_data

(RangeIndex(start=0, stop=10042, step=1),
 0        10514
 1        10513
 2        10488
 3        10542
 4        10506
          ...  
 10037    16957
 10038    16960
 10039    16944
 10040    16945
 10041    16945
 Name: close, Length: 10042, dtype: int64)

In [11]:
Weights = tf.Variable(tf.random.uniform([],minval=3000,maxval=30000))
biases = tf.Variable(tf.zeros(1))

y = Weights*x_data+biases
loss = tf.reduce_mean(tf.square(y-y_data))
optimizer = tf.keras.optimizers.Optimizer(2.0)
train = optimizer.minimize(loss)
init = tf.global_variables_initializer()

TypeError: minimize() missing 1 required positional argument: 'var_list'